In [ ]:
from abc import ABC, abstractmethod
import logging

# Specification
A simpler setup of a component system using *class inheritance* and *abstract base classes (ABC)*. Using the *Runner* as example for a component.

### Requirements
* better overview by fusing config, defaults and implementation
* harmonize API and config usage → same options
* replacing `register` with `__init_subclass__`

### Design
* configs are handled using a special constructor: `from_config`
* possible arguments and default values are set in `__init__` and `from config`
* config options are stored directly in the component

In [ ]:
class Component(ABC):
    _components = {}
    component = "Component"

    @abstractmethod
    def __init__(self):
        pass

    def __init_subclass__(cls, /, label=None):
        """This method is called when a class is subclassed.

        Register a new (sub-)component"""
        if label is None:
            label = cls.__name__

        # register itself with parent
        cls.register(label)(cls)

        if cls.component == "Component":
            # set up new registry for subcomponents
            cls._components = {}
            cls.component = label

    @classmethod
    def register(cls, label):
        """Decorator to register new subcomponents.

        This will allow access to the subcomponent via ``Component[label]``.
        Internally the subcomponents are stored in ``_components``.
        Warning: this is totally unrelated to ABC.register
        """

        def decorator(subcls):
            if label in cls._components:
                logging.warning(
                    f"replacing {cls._components[label]} with {cls} for label '{label}' ({cls.component})."
                )
            cls._components[label] = subcls
            return subcls

        return decorator

    def __class_getitem__(cls, label):
        """Returns the subcomponent."""
        if item is None:
            return cls
        return cls._components[label]

    @classmethod
    @property
    def label(cls):
        """Returns the string label of a subcomponent."""
        for label, item in cls._components.items():  # _components is inherited
            if item == cls:
                return label
        raise NotImplementedError(f"Class {cls} is not registered.")

    @classmethod
    @property
    def labels(cls):
        """Returns the labels of all available subcomponents."""
        return cls._components.keys()

In [ ]:
class Example(Component):
    pass


class SubExample(Example, label="sub"):
    def __init__(self, foo, bar):
        self.foo = foo
        self.bar = bar

    def __repr__(self):
        return f"Example: {self.foo} {self.bar}"

    @classmethod
    def from_config(cls, foo="foo", bar="bar"):
        return cls(foo=foo, bar=bar.upper())

In [ ]:
print(Component.component, Component.labels)
print(SubExample.component, SubExample.labels)

SubExample.from_config()

In [ ]:
print(Component.component, Component.labels)
print(Runner.component, Runner.labels)
print(MPRunner.component, MPRunner.labels)